In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import mahotas as mt
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

def extract_features(image):
    """Extract 36+ features from processed leaf"""
    # 1. Hu Moments (7 shape features)
    moments = cv2.moments(image)
    hu_moments = cv2.HuMoments(moments).flatten()
    
    # 2. Color features (if RGB available)
    if len(image.shape) == 3:
        color_stats = []
        for i in range(3):
            channel = image[:,:,i]
            color_stats.extend([np.mean(channel), np.std(channel)])
    else:
        color_stats = np.zeros(6)
    
    # 3. Haralick texture features (13 features)
    textures = mt.features.haralick(image.astype(np.uint8))
    texture_mean = textures.mean(axis=0)
    
    # Combine all
    features = np.concatenate([hu_moments, color_stats, texture_mean])
    return features

# Create output directory
os.makedirs('features', exist_ok=True)

# Extract features
feature_list = []
labels = []

input_dir = 'data/processed'
print("🔄 Processing folders in:", input_dir)

for species in os.listdir(input_dir):
    species_path = os.path.join(input_dir, species)
    if os.path.isdir(species_path):
        print(f"📁 Processing: {species}")
        img_count = 0
        for img_file in os.listdir(species_path):
            if img_file.lower().endswith(('.png', '.jpg')):
                img_path = os.path.join(species_path, img_file)
                img = cv2.imread(img_path, 0)
                if img is not None:
                    features = extract_features(img)
                    feature_list.append(features)
                    labels.append(species)
                    img_count += 1
        print(f"   ✅ {img_count} images")
    else:
        print(f"⏭️  Skipping: {species} (not folder)")

# Save
if feature_list:
    X = np.array(feature_list)
    y = np.array(labels)
    df_features = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(X.shape[1])])
    df_features['species'] = y
    df_features.to_csv('features/plant_features.csv', index=False)
    
    print("\n" + "="*60)
    print(f"✅ SUCCESS! {X.shape[0]} samples, {X.shape[1]} features")
    print(f"💾 Saved: features/plant_features.csv")
    print("="*60)
    print(df_features.head())
else:
    print("❌ No images found! Check data/processed/ structure")


NameError: name 'os' is not defined